# Regression of Used Car Prices Dataset
Input files - [Downloaded here](https://www.kaggle.com/competitions/playground-series-s4e9/data)

Original dataset - [here](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)
## Development Notes/Ideas


## Libraries

In [1]:
### libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from openfe import OpenFE, tree_to_formula, transform

## Load and Preview Data
### Data Description
- Brand & Model: Identify the brand or company name along with the specific model of each vehicle.
- Model Year: Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.
- Mileage: Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.
- Fuel Type: Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.
- Engine Type: Understand the engine specifications, shedding light on performance and efficiency.
- Transmission: Determine the transmission type, whether automatic, manual, or another variant.
- Exterior & Interior Colors: Explore the aesthetic aspects of the vehicles, including exterior and interior color options.
- Accident History: Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.
- Clean Title: Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.
- Price: Access the listed prices for each vehicle, aiding in price comparison and budgeting.

In [2]:
### load data
train_raw = pd.read_csv('01_Data/train.csv')
test_raw=pd.read_csv('01_Data/test.csv')
original_raw = pd.read_csv('01_Data/used_cars.csv')

### data info
train_raw.info()
print("\n")
test_raw.info()
print("\n")
original_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   -

In [3]:
### preview data
train_raw.head(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
### summarise data 
train_raw.describe(include='all')

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
count,188533.000000,188533,188533,188533.000000,188533.000000,183450,188533,188533,188533,188533,186081,167114,1.885330e+05
unique,NaN,57,1897,NaN,NaN,7,1117,52,319,156,2,1,NaN
top,NaN,Ford,F-150 XLT,NaN,NaN,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,NaN
freq,NaN,23088,2945,NaN,NaN,165940,3462,49904,48658,107674,144514,167114,NaN
mean,94266.000000,NaN,NaN,2015.829998,65705.295174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.387802e+04
std,54424.933488,NaN,NaN,5.660967,49798.158076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.881952e+04
min,0.000000,NaN,NaN,1974.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+03
25%,47133.000000,NaN,NaN,2013.000000,24115.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000e+04
50%,94266.000000,NaN,NaN,2017.000000,57785.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.082500e+04
75%,141399.000000,NaN,NaN,2020.000000,95400.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.990000e+04


In [5]:
### summarise data - original
original_raw.describe(include='all')

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
count,4009,4009,4009.000000,4009,3839,4009,4009,4009,4009,3896,3413,4009
unique,57,1898,NaN,2818,7,1146,62,319,156,2,1,1569
top,Ford,M3 Base,NaN,"110,000 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,A/T,Black,Black,None reported,Yes,"$15,000"
freq,386,30,NaN,16,3309,52,1037,905,2025,2910,3413,39
mean,NaN,NaN,2015.515590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,6.104816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1974.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pre-processing

In [6]:
## create new tables for manipulation
train_clean = train_raw.drop('id',axis=1)
test_clean = test_raw.drop('id',axis=1)

## include original data in train data and flag
original_clean = original_raw.replace({' mi\.':'', '\$':'', ',':''}, regex=True)
original_clean['Source'] = 'orig'
train_clean['Source'] = 'syth'
train_clean = pd.concat([train_clean, original_clean], axis=0, ignore_index=True)

test_clean['Source']='syth'

print('train_clean shape: ', train_clean.shape)
print('test_clean shape: ', test_clean.shape)

train_clean shape:  (192542, 13)
test_clean shape:  (125690, 12)


### Fill NULLS

In [7]:
## identify columns with NaN
cols_w_nan = train_clean.columns[train_clean.isna().any()]
print('train: ', cols_w_nan)
print('test: ', test_clean.columns[test_clean.isna().any()])

train_clean[cols_w_nan].isna().sum()

train:  Index(['fuel_type', 'accident', 'clean_title'], dtype='object')
test:  Index(['fuel_type', 'accident', 'clean_title'], dtype='object')


fuel_type       5253
accident        2565
clean_title    22015
dtype: int64

In [8]:
## clean_title - convert to Boolean and assume NaN is 'False' 
train_clean.loc[train_clean.clean_title == 'Yes', 'clean_title'] = True
train_clean.loc[train_clean.clean_title.isna(), 'clean_title'] = False

test_clean.loc[test_clean.clean_title == 'Yes', 'clean_title'] = True
test_clean.loc[test_clean.clean_title.isna(), 'clean_title'] = False

train_clean['clean_title'].value_counts()

clean_title
True     170527
False     22015
Name: count, dtype: int64

In [9]:
## fuel_type - lots of electric vehicles have NaN fuel_type, or have incorrect fuel type
check_fuel = train_clean[(train_clean.engine.str.contains('Electric Fuel'))]
check_fuel['fuel_type'].value_counts(dropna=False)

## impute all electric vehicles, according to 'engine' field with, 'Electric' in fuel_type
rows_w_electric_train = (train_clean.engine.str.contains('Electric Fuel')) | (train_clean.engine == 'Electric')
rows_w_electric_test = (test_clean.engine.str.contains('Electric Fuel')) | (test_clean.engine == 'Electric')
train_clean.loc[rows_w_electric_train, 'fuel_type'] = 'Electric'
test_clean.loc[rows_w_electric_test, 'fuel_type'] = 'Electric'

## impute all Tesla vehicles with  'Electric' in fuel_type
train_clean.loc[(train_clean.brand=='Tesla'), 'fuel_type'] = 'Electric'
test_clean.loc[(test_clean.brand=='Tesla'), 'fuel_type'] = 'Electric'

In [10]:
## check remaining NaN fuel_type - immaterial number left
train_clean[(train_clean.fuel_type.isna())]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,Source
3286,smart,ForTwo Pure,2008,156875,NaN,70.0HP 1.0L 3 Cylinder Engine Gasoline Fuel,2-Speed A/T,Black,Gray,At least 1 accident or damage reported,True,9000,syth
6794,Chevrolet,Corvette Stingray w/2LT,2023,10900,NaN,490.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,Blue,Brown,None reported,True,67900,syth
7302,Dodge,Prius v Three,2012,107000,NaN,134.0HP 1.8L 4 Cylinder Engine Gas/Electric Hy...,1-Speed A/T,Black,Black,None reported,True,9875,syth
7835,Maserati,Grecale Modena,2023,2200,NaN,325.0HP 2.0L 4 Cylinder Engine Gasoline/Mild E...,1-Speed A/T,Black,Black,None reported,True,55000,syth
7920,Toyota,Prius v Three,2016,184421,NaN,134.0HP 1.8L 4 Cylinder Engine Gas/Electric Hy...,2-Speed A/T,Gray,Black,None reported,True,29999,syth
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188368,Ford,Mustang Mach-E Premium,2022,2058,NaN,Standard Range Battery,Automatic CVT,Red,Black,None reported,False,27785,syth
188916,Ford,Mustang Mach-E Select,2023,923,NaN,Standard Range Battery,Automatic,Rapid Red Metallic Tinted Clearcoat,Black Onyx,None reported,False,46998,orig
188989,Kia,EV6 Wind,2022,11945,NaN,111.2Ah / FR 70kW / RR 160kW (697V),Automatic,Glacier,Black,None reported,False,38998,orig
189774,Porsche,Taycan Turbo,2020,21028,NaN,Dual AC Electric Motors,Single-Speed Fixed Gear,BLACK,BEIGE,None reported,False,96900,orig


In [11]:
## imputing mode for remaining NaNs
imp_mode = SimpleImputer(strategy='most_frequent').set_output(transform='pandas')

for col in ['fuel_type', 'accident']:
    train_clean[col] = imp_mode.fit_transform(train_clean[col].to_numpy().reshape(-1, 1))
    test_clean[col] = imp_mode.transform(test_clean[col].to_numpy().reshape(-1, 1))

#check for any remaining NaNs
print('train NaNs: ', train_clean.columns[train_clean.isna().any()])

train NaNs:  Index([], dtype='object')


### Parsing Columns
- Horsepower
- Engine capacity
- Cylinder

In [12]:
for table in [train_clean, test_clean]:
    table['horsepower'] = table['engine'].str.extract(r'(\d+\.\d)HP').astype('float')
    table['engine_capacity'] = table['engine'].str.extract(r'(\d+\.\d)L ').astype('float')
    ## cyclinder can be described as '_ cycliner engine' or 'v_'
    table['cylinders'] = table['engine'].str.extract(r'(\d+) Cylinder ').astype('float')
    table['cylinders'] = table['cylinders'].fillna(table['engine'].str.extract(r' V(\d+) ').squeeze()).astype('float')

train_clean[['engine','horsepower','engine_capacity','cylinders']]

,engine,horsepower,engine_capacity,cylinders
0,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,172.0,1.6,4.0
1,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,252.0,3.9,8.0
2,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,320.0,5.3,8.0
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,420.0,5.0,8.0
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,208.0,2.0,4.0
...,...,...,...,...
192537,6.0L W12 48V PDI DOHC Twin Turbo,NaN,6.0,NaN
192538,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,349.0,3.0,6.0
192539,Electric,NaN,NaN,NaN
192540,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,450.0,3.5,6.0


In [13]:
# remaining number if NaNs
train_clean[['engine','horsepower','engine_capacity','cylinders']].isna().sum()

engine                 0
horsepower         34069
engine_capacity    14796
cylinders          22595
dtype: int64

In [14]:
## REMOVED
# ## imputing median for remaining NaNs
# imp_median = SimpleImputer(strategy='median').set_output(transform='pandas')

# for col in ['horsepower', 'engine_capacity', 'cylinders']:
#     train_clean[col] = imp_median.fit_transform(train_clean[col].to_numpy().reshape(-1, 1))
#     test_clean[col] = imp_median.transform(test_clean[col].to_numpy().reshape(-1, 1))

# #check for any remaining NaNs
# print('train NaNs: ', train_clean.columns[train_clean.isna().any()])

### Fix Column DataTypes

In [15]:
## convert numerical columns to float
for col in ['milage', 'price']:
    train_clean[col] = train_clean[col].astype('float')

for col in ['milage']:
    test_clean[col] = test_clean[col].astype('float')

## convert clean_title to boolean
train_clean['clean_title'] = train_clean['clean_title'].astype('bool')
test_clean['clean_title'] = test_clean['clean_title'].astype('bool')

## convert remaining object columns to category
for col in train_clean.columns[train_clean.dtypes==object]:
    train_clean[col] = train_clean[col].astype('category')

for col in test_clean.columns[test_clean.dtypes==object]:
    test_clean[col] = test_clean[col].astype('category')

### Feature Engineering
- OpenFE

In [16]:
# target & feature split
train_y = train_clean[['price']]
train_x = train_clean.drop('price', axis=1)
test_x = test_clean
train_x['clean_title'] = train_x['clean_title'].astype('category')  # OpenFE doesn't like Boolean
test_x['clean_title'] = test_x['clean_title'].astype('category')

# feature generation
ofe = OpenFE()
ofe.fit(data=train_x, label=train_y, n_jobs=6)
None

The number of candidate features is 797
Start stage I selection.


100%|██████████| 24/24 [00:16<00:00,  1.43it/s]


195 same features have been deleted.
Meet early-stopping in successive feature-wise halving.


100%|██████████| 24/24 [01:53<00:00,  4.74s/it]


The number of remaining candidate features is 589
Start stage II selection.


100%|██████████| 24/24 [00:56<00:00,  2.35s/it]


Finish data processing.
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.286692 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71451
[LightGBM] [Info] Number of data points in the train set: 154033, number of used features: 604


In [17]:
# OpenFE recommends a list of new features. We include the top 50 generated features 
train_x, test_x = transform(train_x, test_x, ofe.new_features_list[:50], n_jobs=4)
# relabel autoFE with more informative labels
autoFE_labels = []
for feature in ofe.new_features_list[:50]:
    feature = tree_to_formula(feature).replace(',','|')
    autoFE_labels.append(feature)
data_temp = train_clean.drop('price', axis=1)
train_x.columns = data_temp.columns.to_list() + autoFE_labels
test_x.columns = data_temp.columns.to_list() + autoFE_labels

In [21]:
print("The top 20 generated features are")
for i, feature in enumerate(ofe.new_features_list[:20]):
    print(i+1, tree_to_formula(feature))

The top 20 generated features are
1 (milage-model_year)
2 (milage/engine_capacity)
3 (milage/horsepower)
4 (milage/cylinders)
5 GroupByThenMean(milage,engine)
6 (milage+model_year)
7 round(milage)
8 GroupByThenMean(horsepower,model)
9 GroupByThenRank(model_year,clean_title)
10 (milage/model_year)
11 (model_year+engine_capacity)
12 (model_year+cylinders)
13 GroupByThenRank(milage,clean_title)
14 Combine(transmission,int_col)
15 GroupByThenStd(horsepower,int_col)
16 CombineThenFreq(model,ext_col)
17 GroupByThenRank(cylinders,ext_col)
18 GroupByThenRank(milage,model)
19 GroupByThenMean(milage,model_year)
20 GroupByThenRank(milage,engine)


In [24]:
# remove decimal place from 'combine...' categorical columns that were featured engineered by openFE
for col_combine in train_x.filter(regex='^Combine\(').columns:
    train_x[col_combine] = train_x[col_combine].astype('int')
    train_x[col_combine] = train_x[col_combine].astype('category')

    test_x[col_combine] = test_x[col_combine].astype('int')
    test_x[col_combine] = test_x[col_combine].astype('category')  

## Save data

In [25]:
train_clean.to_pickle('train_clean.pkl')
test_clean.to_pickle('test_clean.pkl')

train_fe = pd.concat([train_x,train_y], axis=1)
train_fe.to_pickle('train_fe.pkl')
test_x.to_pickle('test_fe.pkl')